## 1. Extract faces from pictures of people 
### Instrutions:
- Place photos of people (one face visible) in the folder called "./people"
- Replace my photo titled "Avi.jpg" with a piture of your face for testing on a webcam
- Faces are extracted using the haarcascade_frontalface_default detector model
- Extracted faces are placed in the folder called "./group_of_faces"
#### We are extracting the faces needed for our one-shot learning model, it will load 5 extracted faces

In [2]:
from os import listdir
from os.path import isfile, join
import cv2

face_detector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

mypath = "./people/"
image_file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print("Collected image names")

for image_name in image_file_names:
    person_image = cv2.imread(mypath+image_name)
    face_info = face_detector.detectMultiScale(person_image, 1.3, 5)
    for (x,y,w,h) in face_info:
        face = person_image[y:y+h, x:x+w]
        roi = cv2.resize(face, (128, 128), interpolation = cv2.INTER_CUBIC)
    path = "./group_of_faces/" + "face_" + image_name 
    cv2.imwrite(path, roi)
    cv2.imshow("face", roi)
    
    cv2.waitKey(0)
cv2.destroyAllWindows()

Collected image names


## 2. Load our VGGFaceModel 
- This block of code defines the VGGFace model (which we use later) and loads the model

In [3]:
import numpy as np
import cv2

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir

def preprocess_image(image_path):
    """Loads image from path and resizes it"""
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

from tensorflow.keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

model = vgg_face_descriptor

print("Model Loaded")

Model Loaded


## 3. Test model using your Webcam
This code looks up the faces you extracted in the "group_of_faces" folder and uses the similarity (Cosine Similarity) to detect which faces is most similar to the one being extracted with your webcam.

In [ ]:
people_pictures = "./group_of_faces/"

all_people_faces = dict()

for file in listdir(people_pictures):
    if file.endswith(('.jpg', '.jpeg', '.png')):
        person_face, extension = file.rsplit(".", 1)
        image_path = join(people_pictures, file)

        all_people_faces[person_face] = model.predict(preprocess_image(image_path))[0, :]
        print(all_people_faces)

print("Face representations retrieved successfully")

def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

#Open Webcam
cap = cv2.VideoCapture(0) 

while(True):
    ret, img = cap.read()
    faces = face_detector.detectMultiScale(img, 1.3, 5)

    for (x,y,w,h) in faces:
        if w > 100:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            detected_face = img[int(y):int(y+h), int(x):int(x+w)]
            detected_face = cv2.resize(detected_face, (224, 224))

            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255

            captured_representation = model.predict(img_pixels)[0,:]

            found = 0
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]

                similarity = findCosineSimilarity(representation, captured_representation)
                if(similarity < 0.30):
                    cv2.putText(img, person_name[5:], (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    found = 1
                    break

            #connect face and text
            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255, 0, 0),1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255, 0, 0),1)

            if(found == 0):
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('img',img)

    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 26ms/step
{'face_Avi': array([ 5.6508474,  5.6855097,  3.233804 , ..., -2.159198 ,  0.7454784,
        0.3207671], dtype=float32)}
1/1 [==============================] - 0s 14ms/step
{'face_Avi': array([ 5.6508474,  5.6855097,  3.233804 , ..., -2.159198 ,  0.7454784,
        0.3207671], dtype=float32), 'face_Obama': array([ 3.2553482 , -1.9113852 , -1.0515271 , ..., -1.9953029 ,
        1.6405779 ,  0.61411196], dtype=float32)}
1/1 [==============================] - 0s 12ms/step
{'face_Avi': array([ 5.6508474,  5.6855097,  3.233804 , ..., -2.159198 ,  0.7454784,
        0.3207671], dtype=float32), 'face_Obama': array([ 3.2553482 , -1.9113852 , -1.0515271 , ..., -1.9953029 ,
        1.6405779 ,  0.61411196], dtype=float32), 'face_Trump': array([ 0.9752423 , -0.38723707, -1.9627297 , ..., -0.8637647 ,
       -0.4270401 , -1.1794107 ], dtype=float32)}
1/1 [==============================] - 0s 13ms/step
{'face_Avi': array([ 5.6508474,  5.6855097,  